# Matemática com palavras (vetores TF-IDF)

Este capítulo cobre
* Contando palavras, n -gramas e frequências de termos para analisar o significado
* Previsão de probabilidades de ocorrência de palavras com a Lei de Zipf
* Representando textos em linguagem natural como vetores
* Encontrar documentos relevantes em uma coleção de texto usando frequências de documentos
* Estimando a similaridade de pares de documentos com similaridade de cosseno

## 1. Bag Of Words

Vejamos um exemplo em que contar ocorrências de palavras é útil:

In [ ]:
import spacy
spacy.cli.download("en_core_web_sm") # download do modelo de lingua inglesa
nlp = spacy.load("en_core_web_sm") # carregando o modelo de lingua inglesa na variavel nlp
sentence = ('It has also arisen in criminal justice, healthcare, and '
     'hiring, compounding existing racial, economic, and gender biases.')
doc = nlp(sentence) # Processamento da sentença
tokens = [token.text for token in doc] #extração de tokens
tokens



✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


['It',
 'has',
 'also',
 'arisen',
 'in',
 'criminal',
 'justice',
 ',',
 'healthcare',
 ',',
 'and',
 'hiring',
 ',',
 'compounding',
 'existing',
 'racial',
 ',',
 'economic',
 ',',
 'and',
 'gender',
 'biases',
 '.']

In [ ]:
# fazendo o mesmo escremento porém em portugues

spacy.cli.download("pt_core_news_sm")
pt = spacy.load("pt_core_news_sm")
frase = ("O importante é não parar de questionar; a curiosidade tem sua própria razão de existir") # citação de Einstein
br = pt(frase)
t1 = [token.text for token in br]
t1

✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_sm')


['O',
 'importante',
 'é',
 'não',
 'parar',
 'de',
 'questionar',
 ';',
 'a',
 'curiosidade',
 'tem',
 'sua',
 'própria',
 'razão',
 'de',
 'existir']

In [ ]:
#contando a ocorrencia de tokens
from collections import Counter

bag_of_words = Counter(tokens)
bag_of_words

Counter({'It': 1,
         'has': 1,
         'also': 1,
         'arisen': 1,
         'in': 1,
         'criminal': 1,
         'justice': 1,
         ',': 5,
         'healthcare': 1,
         'and': 2,
         'hiring': 1,
         'compounding': 1,
         'existing': 1,
         'racial': 1,
         'economic': 1,
         'gender': 1,
         'biases': 1,
         '.': 1})

In [ ]:
# Counter é usado para contar como o proprio nome diz ora pois KKKKKK
# Um exemplo básico é que le conta as ocorrencias de listas


minha_lista = [1, 1, 2, 1, 2, 3, 4, 1, 0, 2, 5, 6, 7, 8, 9, 1]
print(Counter(minha_lista))
print(Counter(minha_lista).items()) # .items() retorna um objeto de visualização aplicado a dicionários


Counter({1: 5, 2: 3, 3: 1, 4: 1, 0: 1, 5: 1, 6: 1, 7: 1, 8: 1, 9: 1})
dict_items([(1, 5), (2, 3), (3, 1), (4, 1), (0, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1)])


In [ ]:
# rankeando os tokens através de sua quantidade de ocorrencias
bag_of_words.most_common(5) # 5 = apenas os cinco primeiros melhores rankeados

[(',', 5), ('and', 2), ('It', 1), ('has', 1), ('also', 1)]

Pandas Series é uma estrutura de dados eficiente para armazenar as contagens de tokens, incluindo as 2 tuplas do most_common método.

In [ ]:
import pandas as pd
most_common = dict(bag_of_words.most_common()) # transforma a tupla em dicionario
counts = pd.Series(most_common) # passa o dicionário como Series do pandas
counts

,              5
and            2
It             1
has            1
also           1
arisen         1
in             1
criminal       1
justice        1
healthcare     1
hiring         1
compounding    1
existing       1
racial         1
economic       1
gender         1
biases         1
.              1
dtype: int64

In [ ]:
len(counts) # gera o numero de tokens unicos no nosso vocabulario

18

In [ ]:
counts.sum()


23

In [ ]:
len(tokens)

23

In [ ]:
counts / counts.sum()

,              0.217391
and            0.086957
It             0.043478
has            0.043478
also           0.043478
arisen         0.043478
in             0.043478
criminal       0.043478
justice        0.043478
healthcare     0.043478
hiring         0.043478
compounding    0.043478
existing       0.043478
racial         0.043478
economic       0.043478
gender         0.043478
biases         0.043478
.              0.043478
dtype: float64

In [ ]:
counts['justice']

1

In [ ]:
counts['justice'] / counts.sum()

0.043478260869565216

A frequência normalizada do termo "justice" na frase de abertura deste artigo da Wikipedia é de cerca de 4%

In [ ]:
sentence = "Algorithmic bias has been cited in cases ranging from " \
     "election outcomes to the spread of online hate speech."

tokens = [tok.text for tok in nlp(sentence)]
counts = Counter(tokens)
counts

Counter({'Algorithmic': 1,
         'bias': 1,
         'has': 1,
         'been': 1,
         'cited': 1,
         'in': 1,
         'cases': 1,
         'ranging': 1,
         'from': 1,
         'election': 1,
         'outcomes': 1,
         'to': 1,
         'the': 1,
         'spread': 1,
         'of': 1,
         'online': 1,
         'hate': 1,
         'speech': 1,
         '.': 1})

In [ ]:
!pip install wikipedia


  Preparing metadata (setup.py) ... done
  Created wheel for wikipedia: filename=wikipedia-1.4.0-py3-none-any.whl size=11678 sha256=0851dfa0f284012ba16e722fc1a232a8b170b1219fdc90caa817bb4d90b387a2
  Stored in directory: /root/.cache/pip/wheels/5e/b6/c5/93f3dec388ae76edc830cb42901bb0232504dfc0df02fc50de
Successfully built wikipedia


In [ ]:
import wikipedia

def extrair_texto_wikipedia(titulo, idioma='en'):
    try:
        # Configuração do idioma
        wikipedia.set_lang(idioma)

        # Obtém o conteúdo do artigo
        texto_artigo = wikipedia.page(titulo).content

        return texto_artigo
    except wikipedia.exceptions.PageError:
        return f"Artigo '{titulo}' não encontrado na Wikipedia."

# Exemplo de uso
titulo_artigo = 'Algorithmic Bias'
texto_artigo = extrair_texto_wikipedia(titulo_artigo)

# Imprime o texto do artigo
print(texto_artigo)


Algorithmic bias describes systematic and repeatable errors in a computer system that create "unfair" outcomes, such as "privileging" one category over another in ways different from the intended function of the algorithm.
Bias can emerge from many factors, including but not limited to the design of the algorithm or the unintended or unanticipated use or decisions relating to the way data is coded, collected, selected or used to train the algorithm. For example, algorithmic bias has been observed in search engine results and social media platforms. This bias can have impacts ranging from inadvertent privacy violations to reinforcing social biases of race, gender, sexuality, and ethnicity. The study of algorithmic bias is most concerned with algorithms that reflect "systematic and unfair" discrimination. This bias has only recently been addressed in legal frameworks, such as the European Union's General Data Protection Regulation (2018) and the proposed Artificial Intelligence Act (2021

In [ ]:
# realizando uma solicitação HTTP e obtendo o conteudo do arquivo
import requests

url = ('https://gitlab.com/tangibleai/nlpia2/'
        '-/raw/main/src/nlpia2/ch03/bias_intro.txt')

response = requests.get(url) # solicitação HTTP GET para obter o conteudo do arquivo
response # exibindo a resposta HTTP obtida
# response 200 = OK foi bem sucedido

<Response [200]>

Portanto, resumidamente, o código anterior utiliza a biblioteca requests para fazer uma solicitação HTTP GET à URL fornecida e armazena a resposta resultante na variável response. O conteúdo do arquivo de texto pode ser acessado através do atributo response.text.

In [ ]:
# Obtendo os dados em formato de bytes e texto
bias_intro_bytes = response.content # sequencia de bytes
bias_intro = response.text # conteudo da resposta interpretado como texto

# verificação de igualdade entre a representação de bytes e texto
assert bias_intro_bytes.decode() == bias_intro # .decode() transforma a representação de bytes em strings para comparar
# em resumo, esta assertiva verifica se a representação de bytes convertida para texto é igual ao texto origional

# exibição do conteudo do arquivo de texto
bias_intro


'Algorithmic bias describes systematic and repeatable errors in a computer system that create unfair outcomes, such as privileging one arbitrary group of users over others.\nBias can emerge due to many factors, including but not limited to the design of the algorithm or the unintended or unanticipated use or decisions relating to the way data is coded, collected, selected or used to train the algorithm.\nAlgorithmic bias is found across platforms, including but not limited to search engine results and social media platforms, and can have impacts ranging from inadvertent privacy violations to reinforcing social biases of race, gender, sexuality, and ethnicity.\nThe study of algorithmic bias is most concerned with algorithms that reflect "systematic and unfair" discrimination.\nThis bias has only recently been addressed in legal frameworks, such as the 2018 European Union\'s General Data Protection Regulation.\nMore comprehensive regulation is needed as emerging technologies become incre

O pacote requests retorna um objeto de resposta HTTP contendo os cabeçalhos (em .headers) e o corpo (.text) de uma resposta HTTP.  O arquivo bias_intro.txt dos dados do pacote nlpia2 é um instantâneo de 2023 dos três primeiros parágrafos do artigo da Wikipedia.

In [ ]:
# processamento do conteúdo do arquivo de texto
tokens = [tok.text for tok in nlp(bias_intro)] # gerando tokens
counts = Counter(tokens) # contagem de cada token
counts # exibição da contagem


Counter({'Algorithmic': 3,
         'bias': 6,
         'describes': 1,
         'systematic': 2,
         'and': 14,
         'repeatable': 1,
         'errors': 1,
         'in': 10,
         'a': 5,
         'computer': 1,
         'system': 1,
         'that': 3,
         'create': 1,
         'unfair': 2,
         'outcomes': 3,
         ',': 35,
         'such': 2,
         'as': 5,
         'privileging': 1,
         'one': 1,
         'arbitrary': 1,
         'group': 1,
         'of': 16,
         'users': 2,
         'over': 1,
         'others': 1,
         '.': 16,
         '\n': 14,
         'Bias': 2,
         'can': 7,
         'emerge': 1,
         'due': 2,
         'to': 15,
         'many': 3,
         'factors': 1,
         'including': 2,
         'but': 3,
         'not': 4,
         'limited': 2,
         'the': 13,
         'design': 3,
         'algorithm': 3,
         'or': 11,
         'unintended': 1,
         'unanticipated': 3,
         'use': 1,
         

In [ ]:
# verificando os dez mais comuns
counts.most_common(10)

[(',', 35),
 ('of', 16),
 ('.', 16),
 ('to', 15),
 ('and', 14),
 ('\n', 14),
 ('the', 13),
 ('or', 11),
 ('in', 10),
 ('can', 7)]

In [ ]:
# olhando os tokens menos comuns pode ser mais util
counts.most_common()[-5:]

[('programs', 1), ('inputs', 1), ('between', 1), ('same', 1), ('service', 1)]

In [ ]:
# processamento do texto em linhas não vazias
docs = [nlp(s) for s in bias_intro.split('\n') if s.strip()]
counts = []

for doc in docs:
  counts.append(Counter([t.text.lower() for t in doc])) # para cada token no documento, extrai o texto em minuscula e adiciona na lista

# criação de um dataframe a partir das contagens
df = pd.DataFrame(counts)

# preenchimento de valores ausentes e conversão de tipos
df = df.fillna(0).astype(int) # substitui valores ausentes (NaN) por 0, e converte para o tipo inteiro
len(df)

16

* `bias_intro.split('\n')`: Divide ot texto bias_intro em linhas usando a quebra de linha como delimitador

* `if s.strip()`: filtra as linhas não vazias, garantindo que linhas contendo apenas espaços em brancos sejam ignoradas.


Em resumo, este código usa o spaCy para processar linhas não vazias de um arquivo de texto, realiza a contagem de ocorrências de palavras em cada linha e armazena essas contagens em um DataFrame Pandas. O resultado é um DataFrame que mostra a distribuição de palavras em cada linha do texto.

In [ ]:
df.head()

,algorithmic,bias,describes,systematic,and,repeatable,errors,in,a,computer,...,there,no,examine,network,interrelated,programs,inputs,between,same,service
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,1,0,0,3,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,1,0,1,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df.iloc[10]

algorithmic    0
bias           0
describes      0
systematic     0
and            2
              ..
programs       0
inputs         0
between        0
same           0
service        0
Name: 10, Length: 246, dtype: int64

In [ ]:
# criação da lista docs_tokens
docs_tokens = []

# extração de tokens
for doc in docs:
  docs_tokens.append([tok.text.lower() for tok in nlp(doc.text)]) # Para cada token no documento, extrai o texto do token em minusculas e adiciona a lista

len(docs_tokens[0])

27

Em resumo, este código cria uma lista chamada docs_tokens, onde cada elemento é uma lista de tokens de um documento processado. Em seguida, o código exibe o número de tokens no primeiro documento processado. Esse tipo de manipulação de tokens pode ser útil para análise de texto, contagem de palavras ou outras tarefas relacionadas ao processamento de linguagem natural.








In [ ]:
all_doc_tokens = [] # inicializa uma lista vazia que será usada para armazenar todos os tokens de todos os documentos

# adição de tokens
for tokens in docs_tokens:
  # adiciona todos os elementos da lista de tokens atual
  all_doc_tokens.extend(tokens) # extend é utilizado para adicionar os elementos individualmente

len(all_doc_tokens) # comprimento total de tokens

482

Seu léxico ou vocabulário é uma lista de todos os tokens exclusivos do seu corpus

In [ ]:
# criação do conjunto de vocabulário
vocab = set(all_doc_tokens) # cria um conjunto contendo elementos unicos, dado que um conjunto não permite duplicatas

# Ordenação do vocabulário
vocab = sorted(vocab) # ordena as palavras unicas no vocab em ordem alfabetica

# exibição do tamanho do vocab
len(vocab)

# relação entre o numero total de tokens e o tamanho do vocab:
len(all_doc_tokens) / len(vocab)

1.9593495934959348

Em resumo, este código cria um vocabulário único a partir de uma lista de tokens, ordena esse vocabulário e exibe o tamanho do vocabulário. Além disso, ele calcula a média de quantas vezes uma palavra única é usada em média nos documentos, fornecendo uma métrica de diversidade lexical.

In [ ]:
# exibição do tamanho do vocabulario
len(vocab) # numero total de palavras unicas presentes nos documentos

246

In [ ]:
vocab[:10]

['"', "'s", ',', '-', '.', '2018', ';', 'a', 'ability', 'accurately']

In [ ]:
# inicialização da lista
count_vectors = []

# criação de vetores de contagem para cada documento
for tokens in docs_tokens:
  count_vectors.append(Counter(tokens)) # Para cada lista de tokens em um documento, cria um objeto counter que conta a ocorrência de cada palavra na lista

# criação do dataframe pandas
tf = pd.DataFrame(count_vectors)

# transposição e ordenação do DataFrame
tf = tf.T.sort_index().T # tf.T -> substitui linhas por colunas, ordenas as colunas em ordem alfabetica, e depois volta a posição origional (WTF)

# preenchimento de valores ausentes e conversão de tipos
tf = tf.fillna(0).astype(int)

# exibição da matriz de contagem
tf



,"""",'s,",",-,.,2018,;,a,ability,accurately,...,way,ways,website,when,which,who,with,within,world,wrongful
0,0,0,1,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,3,0,1,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
2,0,0,5,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,2,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,1,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,0,0,4,0,1,0,0,0,1,0,...,0,1,0,0,1,0,1,0,1,0
7,0,0,3,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,1,2,1,1,0,2,1,0,0,...,0,0,0,0,0,1,0,0,0,0
9,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Em resumo, este código realiza a contagem de ocorrências de palavras em cada documento, cria uma matriz de contagem (Count Vectorization) representando essas contagens e armazena essa matriz em um DataFrame Pandas chamado tf.

### Usando CountVectorizer

In [ ]:
# Exemplo básico do uso do CountVectorizer:

from sklearn.feature_extraction.text import CountVectorizer

# documentos de exemplo
documents = [
    "Este é um exemplo de texto.",
    "O segundo documento é ainda mais simples.",
    "E o terceiro documento é o último exemplo."
]

# Inicializando essa bosta:
vectorizer = CountVectorizer()

# ajustando e transformand os documentos
X = vectorizer.fit_transform(documents)

# Obtendo o vocabulário (palavras unicas)
vocabulary = vectorizer.get_feature_names_out()

# Convertendo a matriz esparsa para uma matriz densa (para visualização)
matriz_contagens = X.toarray()

# exibindo os resultados
print("Vocabulario:")
print(vocabulary)

print("\n Matriz de contagens: ")
print(matriz_contagens)

Vocabulario:
['ainda' 'de' 'documento' 'este' 'exemplo' 'mais' 'segundo' 'simples'
 'terceiro' 'texto' 'um' 'último']

 Matriz de contagens: 
[[0 1 0 1 1 0 0 0 0 1 1 0]
 [1 0 1 0 0 1 1 1 0 0 0 0]
 [0 0 1 0 1 0 0 0 1 0 0 1]]


-------------
Voltando ao nosso modelo

In [ ]:
# Usando sklearn para calcular vetores de contagem de palavras
from sklearn.feature_extraction.text import CountVectorizer
corpus = [doc.text for doc in docs]
vectorizer = CountVectorizer()
count_vectors = vectorizer.fit_transform(corpus)  # transforma o documento em matriz de contagem
print(count_vectors.toarray()) #


[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 1 0 0]]


In [ ]:
# obtendo o vocabulario (palavras unicas)
cu = vectorizer.get_feature_names_out()
print(cu)

['2018' 'ability' 'accurately' 'across' 'addressed' 'advanced' 'algorithm'
 'algorithmic' 'algorithms' 'also' 'an' 'analysis' 'and' 'anticipated'
 'application' 'arbitrary' 'are' 'arisen' 'arrests' 'as' 'audiences'
 'authority' 'barrier' 'be' 'because' 'become' 'been' 'behavior' 'being'
 'between' 'bias' 'biases' 'but' 'by' 'can' 'cannot' 'cases' 'certain'
 'change' 'cited' 'coded' 'collected' 'color' 'complexity' 'compounding'
 'comprehensive' 'computer' 'concerned' 'considered' 'contexts' 'create'
 'criminal' 'cultural' 'darker' 'data' 'datasets' 'decisions' 'describes'
 'design' 'discovering' 'discrimination' 'displace' 'due' 'easily'
 'economic' 'election' 'emerge' 'emerging' 'engine' 'enter' 'errors'
 'ethnicity' 'european' 'even' 'examine' 'existing' 'expand'
 'expectations' 'expertise' 'faces' 'facial' 'factors' 'for' 'found'
 'frameworks' 'from' 'full' 'functioning' 'furthermore' 'gender' 'general'
 'greater' 'group' 'has' 'hate' 'have' 'healthcare' 'hiring' 'human'
 'identify'

### Similaridade do cosseno

<img src="https://wikimedia.org/api/rest_v1/media/math/render/svg/bc7579979ab78d41b90f2cb517d18229d680dd11">

* Produto escalar entre os vetores
* norma euclidiana de x e y

In [ ]:
import numpy as np

def similaridade_cosseno(vetor_a, vetor_b):
    # Calcula o produto escalar
    produto_escalar = np.dot(vetor_a, vetor_b)

    # Calcula as normas euclidianas dos vetores
    norma_a = np.linalg.norm(vetor_a)
    norma_b = np.linalg.norm(vetor_b)

    # Calcula a similaridade do cosseno
    similaridade = produto_escalar / (norma_a * norma_b)

    return similaridade

# Exemplo de uso
vetor1 = np.array([1, 2, 3])
vetor2 = np.array([3, -5, 6])

resultado = similaridade_cosseno(vetor1, vetor2)
print(f"A similaridade do cosseno entre os vetores é: {resultado}")


A similaridade do cosseno entre os vetores é: 0.35138211074996695


In [ ]:
# norma euclidiana
def norma_euclidiana(vector):
  euclid = np.linalg.norm(vector)
  return euclid

vector = [1, 2, 4, 6, 8, 10, 9]
print(norma_euclidiana(vector))

17.378147196982766


Para vetores de documentos de PNL que possuem semelhança de cosseno próxima a 1, você sabe que os documentos usam palavras semelhantes em proporções semelhantes. Portanto, os documentos cujos vetores de documentos estão próximos uns dos outros provavelmente estão falando sobre a mesma coisa.

## 2. Zipf's Law

In [ ]:
import nltk
nltk.download('brown') # download do corpus brown
from nltk.corpus import brown # Importa o modelo brown do corpus NLTK
brown.words()[:10] # .words() obtem uma lista de todas as palavras no corpus Brown

[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Unzipping corpora/brown.zip.


['The',
 'Fulton',
 'County',
 'Grand',
 'Jury',
 'said',
 'Friday',
 'an',
 'investigation',
 'of']

In [ ]:
brown.tagged_words()[:10]
# tagged_words() -> Obtem uma lista de tuplas onde cada tupla contém uma palavra e sua categoria gramatical

[('The', 'AT'),
 ('Fulton', 'NP-TL'),
 ('County', 'NN-TL'),
 ('Grand', 'JJ-TL'),
 ('Jury', 'NN-TL'),
 ('said', 'VBD'),
 ('Friday', 'NR'),
 ('an', 'AT'),
 ('investigation', 'NN'),
 ('of', 'IN')]

In [ ]:
# quantidade de palavras
len(brown.words())

1161192

In [ ]:
from collections import Counter
puncs = set((',', '.', '--', '-', '!', '?',
     ':', ';', '``', "''", '(', ')', '[', ']'))

word_list = (x.lower() for x in brown.words() if x not in puncs)
token_counts = Counter(word_list)
token_counts.most_common(10)

[('the', 69971),
 ('of', 36412),
 ('and', 28853),
 ('to', 26158),
 ('a', 23195),
 ('in', 21337),
 ('that', 10594),
 ('is', 10109),
 ('was', 9815),
 ('he', 9548)]

In [ ]:
# Python code to illustrate
# working of try()
def divide(x, y):
	try:
		# Floor Division : Gives only Fractional Part as Answer
		result = x // y
		print("Yeah ! Your answer is :", result)
	except ZeroDivisionError:
		print("Sorry ! You are dividing by zero ")

# Look at parameters and note the working of Program
divide(3, 2)


Yeah ! Your answer is : 1


## Inverse Document Frequency (IDF)

Primeiro, vamos obter a contagem total de palavras de cada documento do seu corpus:

In [ ]:
DATA_DIR = ('https://gitlab.com/tangibleai/nlpia/'
             '-/raw/master/src/nlpia/data')
url = DATA_DIR + '/bias_discrimination.txt'
bias_discrimination = requests.get(url).content.decode()
intro_tokens = [token.text for token in nlp(bias_intro.lower())]
disc_tokens = [token.text for token in nlp(bias_discrimination.lower())]
intro_total = len(intro_tokens)
intro_total

498

In [ ]:
disc_total = len(disc_tokens)
disc_total

9627